Get CIK list of all investment companies

In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv 
import edgar



df_MF = pd.read_csv(r'https://goz39a.s3.eu-central-1.amazonaws.com/13F/CIK_list.csv')
df_MF['Entity Name'] = df_MF['Entity Name'].str.lower()
#[i.find(isin(t)) for i in df_MF.loc[df_MF['Entity Name']]]



Add zeros to CIK so that they have 10 digits

In [6]:
CIK = df_MF['CIK Number'][0:500]
A = [5*str(0)+str(cik) for cik in CIK if len(str(cik)) ==5]
B = [6*str(0) + str(cik) for cik in CIK if len(str(cik)) ==4]
CIK = A+B

download all filings from 2020-2021

In [328]:
edgar.download_index('c:/Users/michi/Desktop', 2020, skip_all_present_except_last=False)

select the URL to the edgar archive for the 13F-HR filings of 2020 until q1 of 2021 

In [17]:
def get_urls(path_download_index):
    with open(path_download_index,'r') as file:
        tsvreader = csv.reader(file, delimiter="|")
        urls= []
        file = []
        for line in tsvreader:
            if line[1:][1] == '13F-HR':
                adress=line[-1]
                url =  'https://www.sec.gov/Archives/' + adress
                holding = line[1]
                urls.append((holding.lower(), url))
    return urls


In [159]:
urls_2020_1 = get_urls('C:\\Users\\michi\\Desktop\\2020-QTR1.tsv')
urls_2020_2 = get_urls('C:\\Users\\michi\\Desktop\\2020-QTR2.tsv')
urls_2020_3 = get_urls('C:\\Users\\michi\\Desktop\\2020-QTR3.tsv')
urls_2020_4 = get_urls('C:\\Users\\michi\\Desktop\\2020-QTR4.tsv')
urls_2021 = get_urls('C:\\Users\\michi\\Desktop\\2021-QTR1.tsv')
    

Select only investment companies with 13F-HR filings in all quarters

In [160]:
urls_2020_1names, _ = zip(*urls_2020_1)
urls_2020_2names, _ = zip(*urls_2020_2)
urls_2020_3names, _ = zip(*urls_2020_3)
urls_2020_4names, _ = zip(*urls_2020_4)

companiesBoth = [i for i in urls_2021 if i[0] in (urls_2020_1names and urls_2020_2names and urls_2020_3names and urls_2020_4names)]


In [161]:
len(companiesBoth)

5168

From this, only select biggest companies 

List of biggest investment companies from https://mutualfunddirectory.org/latest-directory-ranking-here/


In [163]:

URL = 'https://mutualfunddirectory.org/latest-directory-ranking-here/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table', class_="tablepress-id-109")
for company in table.find_all('tbody'):
    rows= company.find_all('tr')
    companies =[]
    for row in rows:
        companies.append(row.find('td', class_="column-2").text)

companies = companies[:-1]
companies = [i.lower() for i in companies]


In [164]:
urls=[]
for element in companiesBoth:
    if len(element[0].split())>1:
        if element[0].split()[0] in companies or element[0].split()[1] in companies:
            urls.append(element[1])
    else:
        if element[0].split()[0] in companies:
            urls.append(element[1])


In [165]:
len(urls)

22

get filing detail page on Edgar for each filing

In [135]:
import pandas as pd
def parser(urls):
    df = []

    for url in urls:
        my_header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
                              'AppleWebKit/537.11 (KHTML, like Gecko) '
                              'Chrome/23.0.1271.64 Safari/537.11',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                'Accept-Encoding': 'none',
                'Accept-Language': 'en-US,en;q=0.8',
                'Connection': 'keep-alive'}
        #url = 'https://www.sec.gov/Archives/edgar/data/1000742/0001567619-20-003961-index.html'
        page = requests.get(url,  headers=my_header)    
        data = page.text
        soup = BeautifulSoup(data, "lxml")
        identifier = []
        urls = []

        result= soup.find_all('div', attrs={'class':'formGrouping'})[0]
        date = result.find('div',attrs={'class':'info'}).text
        index = soup.find('div',{'id':'secNum'}).text.replace('SEC Accession No. ','').replace('\n','')


        for link in soup.find_all('a'):
            try:
                if 'xml' in link.get('href'):
                    url_to_save = link.get('href')
                    identifier.append(url_to_save)
            except:
                pass

        for item in identifier:

            urls.append('https://www.sec.gov'+item)
        for url in urls:
            try:
                DF_13F = pd.read_html(url)
                DF_13F = DF_13F[-1]
                DF_13F = DF_13F.iloc[2:]
                new_header = DF_13F.iloc[0]
                DF_13F.columns = new_header
                DF_13F = DF_13F.iloc[1:]
                DF_13F['date_reported'] = date
                cik_company = url.split('data/')[1].split('/')[0]
                DF_13F['cik_company'] = cik_company
                DF_13F['index'] = index
                DF_13F = DF_13F[['index','NAME OF ISSUER','TITLE OF CLASS','CUSIP','(x$1000)','PRN AMT','PRN','date_reported','cik_company']]
                df.append(DF_13F)
                print('Filing found for cik %s at %s' %(cik_company,url))

            except:
                print('Error: no information found for cik %s at: %s'%(cik_company,url))
    return pd.concat(df)






dataframe <br>
TODO: add location (city or state) of company + revenue at that time

In [157]:
df=parser(urls)

Error: no information found for 1027796 at: https://www.sec.gov/Archives/edgar/data/1027796/000156761921002265/xslForm13F_X01/primary_doc.xml
Error: no information found for 1027796 at: https://www.sec.gov/Archives/edgar/data/1027796/000156761921002265/primary_doc.xml
Filing found for 1027796 at https://www.sec.gov/Archives/edgar/data/1027796/000156761921002265/xslForm13F_X01/form13fInfoTable.xml
Error: no information found for 1027796 at: https://www.sec.gov/Archives/edgar/data/1027796/000156761921002265/form13fInfoTable.xml
Error: no information found for 102909 at: https://www.sec.gov/Archives/edgar/data/102909/000110465921021334/xslForm13F_X01/primary_doc.xml
Error: no information found for 102909 at: https://www.sec.gov/Archives/edgar/data/102909/000110465921021334/primary_doc.xml
Error: no information found for 102909 at: https://www.sec.gov/Archives/edgar/data/102909/000110465921021334/xslForm13F_X01/infotable.xml
Error: no information found for 102909 at: https://www.sec.gov/Ar

Error: no information found for 801051 at: https://www.sec.gov/Archives/edgar/data/801051/000108514621000293/xslForm13F_X01/infotable.xml
Error: no information found for 801051 at: https://www.sec.gov/Archives/edgar/data/801051/000108514621000293/infotable.xml
Error: no information found for 861177 at: https://www.sec.gov/Archives/edgar/data/861177/000086117721000006/xslForm13F_X01/primary_doc.xml
Error: no information found for 861177 at: https://www.sec.gov/Archives/edgar/data/861177/000086117721000006/primary_doc.xml
Error: no information found for 861177 at: https://www.sec.gov/Archives/edgar/data/861177/000086117721000006/xslForm13F_X01/Information_Table.xml
Error: no information found for 861177 at: https://www.sec.gov/Archives/edgar/data/861177/000086117721000006/Information_Table.xml
Error: no information found for 870481 at: https://www.sec.gov/Archives/edgar/data/870481/000156761921003782/xslForm13F_X01/primary_doc.xml
Error: no information found for 870481 at: https://www.se

In [158]:
df

2                         index                NAME OF ISSUER  \
3    0001567619-21-002265                         ABBVIE INC.   
4    0001567619-21-002265                               AECOM   
5    0001567619-21-002265        ALEXION PHARMACEUTICALS INC.   
6    0001567619-21-002265                ALLSTATE CORPORATION   
7    0001567619-21-002265                   AMBEV SA SPON ADR   
..                          ...                           ...   
181  0000949509-21-000001                VONAGE HOLDINGS CORP   
182  0000949509-21-000001                          WEIBO CORP   
183  0000949509-21-000001                WESTERN DIGITAL CORP   
184  0000949509-21-000001                         WORKDAY INC   
185  0000949509-21-000001                         WORKIVA INC   

2      TITLE OF CLASS      CUSIP (x$1000)   PRN AMT  PRN date_reported  \
3                 COM  00287Y109      668      6233   SH    2021-02-05   
4                 COM  00766T100    23149    465025   SH    2021-02-05   
5                 COM  015351109      633      4049   SH    2021-02-05   
6                 COM  020002101      516      4692   SH    2021-02-05   
7                 COM  02319V103     5035   1645310   SH    2021-02-05   
..                ...        ...      ...       ...  ...           ...   
181  CONVERTIBLE BOND  92886TAJ1    14130  12963000  PRN    2021-02-16   
182  CONVERTIBLE BOND  948596AC5     6799   6974000  PRN    2021-02-16   
183  CONVERTIBLE BOND  958102AP0    17271  17290000  PRN    2021-02-16   
184  CONVERTIBLE BOND  98138HAF8     3351   2000000  PRN    2021-02-16   
185  CONVERTIBLE BOND  98139AAB1     3905   2900000  PRN    2021-02-16   

2   cik_company  
3       1027796  
4       1027796  
5       1027796  
6       1027796  
7       1027796  
..          ...  
181      949509  
182      949509  
183      949509  
184      949509  
185      949509  

[40473 rows x 9 columns]